In [ ]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=11)

In [ ]:
import datetime
import pandas as pd
import numpy as np
from os import listdir
import math

#...assignments
m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
base = 0.05       # Upper or Lower base multiple for prices
expmult = 1.05   # 1.05 is 5% expected multiple. Used for expected sowing price
desired_rom = 1.1 # desired rom to give the target price.

min_rom = 0.85
min_pop = 0.85
min_dte = 45     # no of minimum dte days to determine ohlc filter for min and max

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin
    
#... Functions
#_____________

# gets days to expiry from now onwards
def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format 'yyyymmdd'
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
    # if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = get_dte(expiry)
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

def get_prec(v, base):
    '''gives the precision value
    args:
       (v) as value needing precision in float
       (base) as the base value e.g. 0.05'''
    
    return round(round((v)/ base) * base, -int(math.floor(math.log10(base))))

def grp_opts(df):
    '''Groups options and sorts strikes by puts and calls
    Arg: df as dataframe
    Returns: sorted dataframe'''
    
    gb = df.groupby('right')

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    else:
        df_calls =  pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    else:
        df_puts =  pd.DataFrame([])

    df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
    
    return df

#...Strategies to filter
#------------------------
def filter_kxdte(df):
    '''Filters the strikes by dte*3, and, min of lows for Puts and max of highs for Calls
    Arg: df as dataframe
    Returns: cleansed dfs without the risky strikes'''
    
    df['sfilt'] = [df_ohlc.set_index('symbol').loc[s][:max(min_dte, d*3)].low.min() 
     if g == 'P' 
     else df_ohlc.set_index('symbol').loc[s][:max(min_dte, d*3)].high.max() 
     for s, d, g in zip(df.symbol, df.dte, df.right)]
    
    df = df[((df.right == 'P') & (df.strike < df.sfilt)) | \
            ((df.right == 'C') & (df.strike > df.sfilt))]
    
    # return sorted df of puts and calls    
    df1 = grp_opts(df)
    
    # drop sfilt    
    return df1.drop('sfilt', axis=1)

#-----------------------------

def strat_hilo52(df):
    '''Keeps only options beyond 52 week high and low
    Arg: (df) as dataframe object
    Return: (df_opt) as dataframe with hilo52 options'''
    hilo_mask = ((df.right == 'P') & (df.strike < df.lo52)) | ((df.right == 'C') & (df.strike > df.hi52))
    df_opt = df[hilo_mask].reset_index(drop=True)
    return df_opt

#-----------------------------

def strat_onlyputs(df):
    '''Keep only puts
    Arg: (df) as dataframe object
    Return: (df_opt) without calls'''
    
    return df[df.right=='P'].reset_index()

#...get current positions
#________________________

#... read the account info
ac = ib.accountValues()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#...Harvest preparation
#______________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
df_p['dte'] = [get_dte(d.lastTradeDateOrContractMonth) for d in df_p.contract]

discount = [m for m in map(expPricePct, expiry)]
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]

# ignore data for dte < 3 days. These are as good as gone.
df_h = df_p[df_p.dte > 3].reset_index(drop=True)

hqc = ib.qualifyContracts(*df_h.contract)
df_h = df_h.assign(qual_contract=hqc)

# ...sowing prepration
#_____________________

#... get the lots and margins
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

df_slm['lot'] = df_slm.position/df_slm.Mlot # lots currently held

pos_cols = ['ibSymbol', 'dte', 'position', 'lot', 
 'marketPrice', 'averageCost', 'hvstPrice', 'contract', 'harvestOrder']
df_slm = df_slm[pos_cols]

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()

df2 = pd.merge(df1, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']].set_index('ibSymbol'), left_index=True, right_index=True )

df2['maxlot'] = [float(round(-max_p/lot/mgn)) for lot, mgn in zip(df2.Mlot, df2.TotMgnPerShr)]

df2['remqty'] = np.where(df2.lot - df2.maxlot > 0, df2.lot - df2.maxlot, 0.0)  # remaining quantity

blacklist = list(df2[df2.remqty <= 0].index)

# get remaining lots of partially filled symbols
not_black = df2[df2.remqty > 0][['remqty']].to_dict('dict')
remqtydict = [v for k, v in not_black.items()][0]

In [ ]:
#...build the high-pop-roc dataframe
fs = listdir('./zdata/')

opts = ([f[:-8]+'_opt.pkl' for f in fs if f[-8:] == '_opt.pkl'])
ohlcs = ([f[:-8]+'_ohlc.pkl' for f in fs if f[-8:] == '_opt.pkl'])
unds = ([f[:-8]+'_und.pkl' for f in fs if f[-8:] == '_opt.pkl'])

df_opt = pd.concat([pd.read_pickle('./zdata/'+f) for f in opts], axis=0, sort=True).reset_index(drop=True).sort_values('rom', ascending=False)
df_ohlc = pd.concat([pd.read_pickle('./zdata/'+f).reset_index() for f in ohlcs], axis=0, sort=True)
df_und = pd.concat([pd.read_pickle('./zdata/'+f) for f in unds])

# remove options in black list
df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

# get the lots
df_mlot = df_und[['symbol', 'lot']]
df_opt1 = df_opt.merge(df_mlot, on='symbol', how='inner')

# arrange the columns
cols = ['symbol', 'right', 'expiry', 'dte', 'strike', 'undPrice', 'lo52',  'hi52', 
'stdev', 'volatility', 'margin', 'lot', 'bsmPrice', 'pop', 'rom', 'price', 'option']

df_opt1 = df_opt1[cols]

# take only high rom and pops
df_opt2 = df_opt1[(df_opt1.rom > min_rom) & 
                   (df_opt1.dte > 2) &
                   (df_opt1['pop'] > min_pop)].reset_index(drop=True)

# get the better of Price and bsmPrice for the option - with an expected multiple (expmult)
max_price_bsm = pd.concat([df_opt2.price, df_opt2.bsmPrice], axis=1).max(axis=1)

df_opt2['expPrice'] = get_prec(max_price_bsm*expmult, base)

df_opt3 = filter_kxdte(df_opt2)

In [ ]:
# Make df the dataframe that you want to execute on!
df = df_opt3.copy()   # make this the last dataframe to get the orders placed

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2

contracts = [c for c in df.option]

ass_limit = max(df.strike*df.lot)+100  # assumes how much it will cost if the strike gets assigned (theoretical for NSE)

df = df.assign(qty=pd.concat([round(ass_limit / (df.strike * df.lot)), 
                         round(max_p/df.margin), 
                         pd.Series(5, index=df.index)], axis=1).min(axis=1))
    
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(contracts), \
      len(df.symbol.unique()), sum(df.margin*df.qty), av_funds*max_nlvp, sum(df.expPrice*df.lot*df.qty)))

In [ ]:
# ...review calls and puts
# e.g. for puts from path: C:\Users\kashir\Documents\IBKR\nse\zdata\putswatch.csv
# .....,, or in this path: C:\Users\User\Documents\ibkr\nse\zdata\putswatch.csv (home laptop)

# add tgtPrice based on desired rom
df['tgtPrice'] = get_prec(pd.concat([df.rom, pd.Series(desired_rom, index=df.index)], axis=1).max(axis=1)*df.expPrice/df.rom, base)

df['remqty'] = round(max_p/df.margin)

cols = ['right', 'symbol', 'strike', 'undPrice', 'dte', 'pop', 'rom', 'price', 'expPrice', 'margin', 'lot', 'qty', 'remqty', 'tgtPrice', 'option']
df = df[cols]

# replace remqty with non-blacklist remaining quantities
df = df.set_index('symbol')
df.remqty = np.where(df.index.isin(remqtydict.keys()), df.index.map(remqtydict), df.remqty)
df = df.reset_index()

# Sort the calls and puts by symbol and strikes - to quickly weed out risky options
gb = df.groupby('right')

if 'C' in [k for k in gb.indices]:
    df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    
    # make watchlist
    watchcalls = [('DES', s, 'STK', 'NSE') for s in df_calls.symbol.unique()]
    df_wp = util.df(watchcalls)
    df_wp.to_csv('./zdata/callswatch.csv', index=None, header=False)
else:
    df_calls = pd.DataFrame([]) # empty dataframe

if 'P' in [k for k in gb.indices]:
    df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    
    # make watchlist
    watchputs = [('DES', s, 'STK', 'NSE') for s in df_puts.symbol.unique()]
    df_wp = util.df(watchputs)
    df_wp.to_csv('./zdata/putswatch.csv', index=None, header=False)
else:
    df_puts = pd.DataFrame([]) # empty dataframe

# output the consolidated puts and calls dataframe
df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
df.to_csv('./zdata/check.csv', index=None, header=True)

In [ ]:
# After going through checked.csv, with puts and calls, eliminate risky options
# Save the file as checked.csv

df_final = pd.read_csv('./zdata/checked.csv') # picks up the checked and ready-to-go contracts
cs = [eval(c) for c in df_final.option]  # convert the "quoted strings" from csv back to object
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=tgtPrice) for lot, tgtPrice in zip(df_final.lot, df_final.tgtPrice)]
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(cs), \
      len(df_final.symbol.unique()), sum(df_final.margin), av_funds*max_nlvp, sum(df_final.tgtPrice*df_final.lot*df_final.qty)))